Clickable Histogram of Atmospheric Data (CHAD)

Author: Matthew Niznik (mailto:matthew.niznik9@gmail.com)
Post-Doctoral Associate, RSMAS, University of Miami

The purpose of the Clickable Histogram of Atmospheric Data (CHAD) is to load a 2-D histogram and the data used to generate it in order to create a 2-D scatterplot allowing the user to load the location and time that a particular datapoint in the plot came from using Unidata's Integrated Data Viewer (IDV) (http://www.unidata.ucar.edu/software/idv/).

Input needed:

Binned 2-D histogram data
1-D arrays containing the edges of each histogram bin (i.e. if 10x10, 11 entry long arrays)
3-D [F(x,y,t)] variables (Ex: Precipitation and W(z=500 hPa))
3-D [F(x,y,t)] variables that in lieu of actual values list the bin to which the data belongs
(Ex: PrecBinned(x,y,t) = [0-9] if there are 10 bins)

In [2]:
#--- User Changeable Parameters (and appropriate libraries) ---

#--- Figure Size and Resolution ---
figureXSize = 800
figureYSize = 800
figDPI = 150
maxPlottedInBin = 100

#--- Formatting for Output ---
#--- Basic Help: The number after the decimal point sets the number of decimal points shown in output ---
xFmtStr = "%.2f"
yFmtStr = "%.1f"

#--- Data Resolution Parameters---
Nx = 90
Ny = 45
Nt = 3

import datetime
startDate = datetime.datetime(2005,06,01)

#--- Load Input from URL/Filepath ---
urlToLoadHist = '/Users/niznik/Work/Data/PCH_Data/histOutput_r90x45_3_V2.nc4'
urlToLoadValues = '/Users/niznik/Work/Data/PCH_Data/allVars_r90x45_3.nc4'
var1Name = 'Precipitation'
var2Name = 'W500'
histName = 'HIST'
var1EdgeName = 'PRECBIN'
var2EdgeName = 'W500BIN'
var1BinnedName = 'PRECBINNED'
var2BinnedName = 'W500BINNED'
var1ValueName = 'PREC'
var2ValueName = 'W'
timeValueName = 'time'

#--- Select Regional Subset ---
lonBin = 0
latBin = 5

In [1]:
#--- Importing Libraries (those not needed for defining user variables) ---
from IPython.display import clear_output

import numpy as np
import netCDF4
from subprocess import call
import sys
import time

import matplotlib
from matplotlib import pylab
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

In [3]:
#--- Calculate Supplemental Values---

#--- Data Resolution Parameters ---
resX = 360/Nx
resY = 180/Ny
startHourInS = Nt*(3600./2.)
startDatetime = startDate+datetime.timedelta(0,startHourInS)

In [4]:
#--- Fixing the output so it isn't buffered ---
#--- See: http://stackoverflow.com/questions/29772158/make-ipython-notebook-print-in-real-time ---

oldsysstdout = sys.stdout
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()
sys.stdout = flushfile(sys.stdout)

In [5]:
#--- Load the Data ---
#--- Load Histogram and Bin Edges ---
cdfInHist = netCDF4.Dataset(urlToLoadHist,'r')
hist = np.sum(cdfInHist.variables[histName][:,lonBin,latBin,:,:,:,:,:,:],(0,2,3,5,6))
totalCounts = np.sum(hist)
var1Edges = np.append(cdfInHist.variables[var1EdgeName+'MINS'][0],cdfInHist.variables[var1EdgeName+'MAXS'][:])
var2Edges = np.append(cdfInHist.variables[var2EdgeName+'MINS'][0],cdfInHist.variables[var2EdgeName+'MAXS'][:])
numLonBins = len(cdfInHist.variables['LONBINS'][:])
numLatBins = len(cdfInHist.variables['LATBINS'][:])

#--- Subsetting the timeseries data based on lon/lat bin inputs ---
lonAVMin = lonBin*(Nx/numLonBins)
lonAVMax = lonAVMin+(Nx/numLonBins)
latAVMin = latBin*(Ny/numLatBins)
latAVMax = latAVMin+(Ny/numLatBins)

var1Binned = cdfInHist.variables[var1BinnedName][:,latAVMin:latAVMax,lonAVMin:lonAVMax]
var2Binned = cdfInHist.variables[var2BinnedName][:,latAVMin:latAVMax,lonAVMin:lonAVMax]
cdfInHist.close()
#--- Load Timeseries ---
cdfInValues = netCDF4.Dataset(urlToLoadValues,'r')
var1Values = cdfInValues.variables[var1ValueName][:,latAVMin:latAVMax,lonAVMin:lonAVMax]*86400.
var2Values = cdfInValues.variables[var2ValueName][:,latAVMin:latAVMax,lonAVMin:lonAVMax]
timeValues = cdfInValues.variables[timeValueName][:]
cdfInValues.close()

In [6]:
#--- Determine positions in bins with more entries than maxPlottedInBin ---
plotPositions = []
for v1bin in range(0,hist.shape[0]):
    plotPositions.append([])
    for v2bin in range(0,hist.shape[1]):
        plotPositions[v1bin].append([])
        for yy in range(0,var1Binned.shape[1]):
            for xx in range(0,var1Binned.shape[2]):
                localMatches = numpy.intersect1d(np.where(var1Binned[:,yy,xx] == v1bin)[0],np.where(var2Binned[:,yy,xx] == v2bin)[0])
                plotPositions[v1bin][v2bin] = []
                np.random.shuffle(localMatches)
                if(len(localMatches) != 0):
                    for mm in range(0,min(len(localMatches),maxPlottedInBin)):
                        plotPositions[v1bin][v2bin].append([localMatches[mm],yy,xx])           

In [7]:
#--- Colorbar Parameters ---
minPower = int(np.log10(1./totalCounts))-1
maxPower = 0
levTicks = np.arange(minPower,maxPower+1,1)

In [8]:
#--- Converting histograms to fractional logspace ---
histLog = np.log10(1.0*hist/totalCounts)
histLog = np.where(histLog == -1*np.inf,minPower,histLog)

#--- Creating 1D histograms, converting to fractional logspace ---
histX = np.sum(hist,1)
histY = np.sum(hist,0)
histXLog = np.log10(1.0*histX/np.sum(histX))
histXLog = np.where(histXLog == -1*np.inf,minPower,histXLog)
histYLog = np.log10(1.0*histY/np.sum(histY))
histYLog = np.where(histYLog == -1*np.inf,minPower,histYLog)

#--- Flattening scatter data ---
#pointValuesX = var1Values.flatten()
#pointValuesY = var2Values.flatten()
pointValuesX = var1Values[:]
pointValuesY = var2Values[:]

/Users/niznik/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: RuntimeWarning: divide by zero encountered in log10
  from IPython.kernel.zmq import kernelapp as app


In [9]:
#--- Renaming edge variables for plotting and making copy with
#--- evenly spaced edges (don't want them at first
#--- in the plot so that we can have even spacing)
xBinEdges = var1Edges
yBinEdges = var2Edges
xBinEdgeLen,yBinEdgeLen = xBinEdges.size,yBinEdges.size

xBinEdgeScat,yBinEdgeScat = np.zeros(xBinEdgeLen),np.zeros(yBinEdgeLen)
xBinEdgeScat[:],yBinEdgeScat[:] = xBinEdges[:],yBinEdges[:]
xBinEdgeScatInt = xBinEdgeScat[2]-xBinEdgeScat[1]
yBinEdgeScatInt = yBinEdgeScat[2]-yBinEdgeScat[1]

xBinEdgeScat[0] = xBinEdgeScat[1]-xBinEdgeScatInt
xBinEdgeScat[-1] = xBinEdgeScat[-2]+xBinEdgeScatInt
yBinEdgeScat[0] = yBinEdgeScat[1]-yBinEdgeScatInt
yBinEdgeScat[-1] = yBinEdgeScat[-2]+yBinEdgeScatInt

unlabeledX = np.arange(0,xBinEdgeLen,1)
unlabeledY = np.arange(0,yBinEdgeLen,1)

In [10]:
class showPointData:
    def __init__(self,figure,axes,xData,yData,tData):
        self.figure = figure
        self.axes = axes
        self.xData = xData
        self.yData = yData
        self.tData = tData
        self.xPlotMin = axes.get_xaxis().get_ticklocs()[0]
        self.yPlotMin = axes.get_yaxis().get_ticklocs()[0]
        self.xPlotSpan = axes.get_xaxis().get_ticklocs()[-1]-self.xPlotMin
        self.yPlotSpan = axes.get_yaxis().get_ticklocs()[-1]-self.yPlotMin
        self.xPixFracStart = 0.3
        self.yPixFracStart = 0.3
        self.xPixFracEnd = 0.9
        self.yPixFracEnd = 0.9
        self.cid = figure.canvas.mpl_connect('button_press_event',self)
        self.clicks = 0
        self.lastClickIndex = -1
        print 'Interactive Plot Initialized!'
        print ''
        
    def __call__(self,event):
        if(self.lastClickIndex == -1):
            clear_output()
        figDimArray = self.figure.get_size_inches()*self.figure.get_dpi()
        figXPixels,figYPixels = figDimArray[0],figDimArray[1]
        xClickFrac = (event.x*1.0)/figXPixels
        yClickFrac = (event.y*1.0)/figYPixels
        if(self.xPixFracStart < xClickFrac < self.xPixFracEnd):
            if(self.yPixFracStart < yClickFrac < self.yPixFracEnd):  
                xClickFracInPlot = (xClickFrac-self.xPixFracStart)/(self.xPixFracEnd-self.xPixFracStart)
                yClickFracInPlot = (yClickFrac-self.yPixFracStart)/(self.yPixFracEnd-self.yPixFracStart)
                xPlotVal = self.xPlotMin+xClickFracInPlot*self.xPlotSpan
                yPlotVal = self.yPlotMin+yClickFracInPlot*self.yPlotSpan
                closestDataIndex = findNearestPoint(xPlotVal,yPlotVal,self.xData,self.yData,
                                                   self.xPlotSpan,self.yPlotSpan)
                if(self.lastClickIndex != closestDataIndex):
                    print ('You clicked at X='+
                           str(xFmtStr%xPlotVal)+' Y='+str(yFmtStr%yPlotVal))
                    print ('Nearest data point is X='+
                           str(xFmtStr%self.xData[closestDataIndex])+' Y='+
                           str(yFmtStr%self.yData[closestDataIndex]))
                    print '(Click again to open in IDV)'
                    self.lastClickIndex = closestDataIndex
                else:
                    print 'Opening case in IDV...'
                    caseTimeLatLon = np.where(var1Values == self.xData[closestDataIndex])
                    inputLon = 2.0+(caseTimeLatLon[2][0])*4.0
                    inputLat = 12.0+(caseTimeLatLon[1][0])*4.0
                    inputTime = int(time.mktime((startDatetime+datetime.timedelta(0,
                                    (Nt*3600.)*timeValues[caseTimeLatLon[0][0]])).timetuple()))
                    runIDV(inputLon,inputLat,inputTime)
                    self.lastClickIndex = -1

In [11]:
def runIDV(inputLon,inputLat,inputTime):
    #IDV has an offset from my expected Unix times
    timeUnknownOffset = 2*3600
    adjTime = (inputTime+timeUnknownOffset)*1000
    
    lonOffset = 5.0
    latOffset = 5.0
    timeOffsetBefore = -2*3600*1000
    timeOffsetAfter = 4*3600*1000-timeOffsetBefore
    
    westLon = str(inputLon-lonOffset)
    eastLon = str(inputLon+lonOffset)
    southLat = str(inputLat-latOffset)
    northLat = str(inputLat+latOffset)
    
    startTime = str(adjTime+timeOffsetBefore)
    endTime = str(adjTime+timeOffsetAfter)
    startOffset = str(timeOffsetBefore/(60.*1000.))
    endOffset = str(timeOffsetAfter/(60.*1000.))
    
    currentUnixTime = str(int(time.time()))
    basisBundleFile = './Bundles/'+'PCH_testBundle.xidv'
    tempBundleFile = './Bundles/TempBundles/tempBundle_'+currentUnixTime+'.xidv'
    
    centerLonFiller = '-154.123456789'
    lonLenFiller = '10.123456789'
    minLonFiller = '-159.1851851835'
    maxLonFiller = '-149.0617283945'
    centerLatFiller = '0.135792468'
    latLenFiller = '7.592592592'
    minLatFiller = '-3.660503828'
    maxLatFiller = '3.932088764'
    startTimeFiller = '1117594837000'
    endTimeFiller = '1117616461000'
    startOffsetFiller = '-119.87654321'
    endOffsetFiller = '361.23456789'
    
    call('sed \'s/'+minLonFiller+'/'+westLon+'/\' '+basisBundleFile+' > '+tempBundleFile,shell=True)
    call('sed -i \'.bckp\' \'s/'+maxLonFiller+'/'+eastLon+'/\' '+tempBundleFile,shell=True)
    call('sed -i \'.bckp\' \'s/'+minLatFiller+'/'+southLat+'/\' '+tempBundleFile,shell=True)
    call('sed -i \'.bckp\' \'s/'+maxLatFiller+'/'+northLat+'/\' '+tempBundleFile,shell=True)
    call('sed -i \'.bckp\' \'s/'+centerLonFiller+'/'+str(inputLon)+'/\' '+tempBundleFile,shell=True)
    call('sed -i \'.bckp\' \'s/'+centerLatFiller+'/'+str(inputLat)+'/\' '+tempBundleFile,shell=True)
    call('sed -i \'.bckp\' \'s/'+lonLenFiller+'/'+str(lonOffset*2)+'/\' '+tempBundleFile,shell=True)
    call('sed -i \'.bckp\' \'s/'+latLenFiller+'/'+str(latOffset*2)+'/\' '+tempBundleFile,shell=True)
    call('sed -i \'.bckp\' \'s/'+startTimeFiller+'/'+startTime+'/\' '+tempBundleFile,shell=True)
    call('sed -i \'.bckp\' \'s/'+endTimeFiller+'/'+endTime+'/\' '+tempBundleFile,shell=True)
    call('sed -i \'.bckp\' \'s/'+startOffsetFiller+'/'+startOffset+'/\' '+tempBundleFile,shell=True)
    call('sed -i \'.bckp\' \'s/'+endOffsetFiller+'/'+endOffset+'/\' '+tempBundleFile,shell=True)
    
    call('runIDV -bundle '+tempBundleFile,shell=True)
    call('rm -i '+tempBundleFile,shell=True)
    call('rm '+tempBundleFile+'.bckp',shell=True)
    
    return

In [12]:
def findNearestPoint(xPlotVal,yPlotVal,xData,yData,xPlotSpan,yPlotSpan):
    minDist = 1000000.
    locToRet = -1.
    for tt in range(0,len(yData)):
        xDistNorm = abs(xPlotVal-xData[tt])*(1.0/xPlotSpan)
        yDistNorm = abs(yPlotVal-yData[tt])*(1.0/yPlotSpan)
        totDistNorm = (xDistNorm**2+yDistNorm**2)**0.5
        if(totDistNorm < minDist):
            minDist = totDistNorm
            locToRet = tt
    return locToRet

In [ ]:
%qtconsole

#--- Setting a few final parameters before plotting ---
xLabel = var1Name
yLabel = var2Name

#--- Setting up the figure, creating the palette for the colorbar ---
fig = plt.figure(figsize=((figureXSize*1.0)/figDPI,(figureYSize*1.0)/figDPI),dpi=figDPI)
axarr = fig.add_axes([0.3,0.3,0.7,0.6])
p = axarr.pcolor(xBinEdgeScat,yBinEdgeScat,histLog,vmin=minPower,
                 vmax=maxPower,cmap='Spectral_r')
axarr.cla()

#--- Looping through the data and only scattering as many points as specified ---
for xAxisBin in range(0,hist.shape[0]):
    for yAxisBin in range(0,hist.shape[1]):
        localPlotPositions = plotPositions[xAxisBin][yAxisBin]
        for ee in range(0,len(localPlotPositions)):
            xI = localPlotPositions[ee][2]
            yI = localPlotPositions[ee][1]
            tI = localPlotPositions[ee][0]
            
            locPointValueX = pointValuesX[tI,yI,xI]
            locPointValueY = pointValuesY[tI,yI,xI]
            
            if(locPointValueX < xBinEdgeScat[0]):
                locPointValueX = xBinEdgeScat[0]
            if(locPointValueX > xBinEdgeScat[-1]):
                locPointValueX = xBinEdgeScat[-1]
            if(locPointValueY < yBinEdgeScat[0]):
                locPointValueY = yBinEdgeScat[0]
            if(locPointValueY > yBinEdgeScat[-1]):
                locPointValueY = yBinEdgeScat[-1]
            
            logCount = histLog[xAxisBin,yAxisBin]
            cbPercent = (logCount-minPower)*(1.0/abs(minPower))
            pointColor = pylab.cm.Spectral_r(cbPercent)
            axarr.scatter(locPointValueX,locPointValueY,c=pointColor)

#--- Setting axes ---
axarr.set_xlim(xBinEdgeScat[0],xBinEdgeScat[-1])
axarr.set_xticks(xBinEdgeScat)
axarr.set_xticklabels(xBinEdges[0:xBinEdgeLen],rotation=270)
axarr.tick_params(axis='x',labelsize=8)
#axarr.set_xlabel(xLabel)
axarr.set_ylim(yBinEdgeScat[0],yBinEdgeScat[-1])
axarr.set_yticks(yBinEdgeScat)
axarr.set_yticklabels(yBinEdges[0:yBinEdgeLen],rotation=360)
axarr.tick_params(axis='y',labelsize=8)
#axarr.set_ylabel(yLabel)

#--- Creating gridlines ---
for yy in range(0,len(yBinEdgeScat)):
    axarr.axhline(y=yBinEdgeScat[yy],color='#000000')
for xx in range(0,len(xBinEdgeScat)):
    axarr.axvline(x=xBinEdgeScat[xx],color='#000000')
    
axarr.set_title('Test',fontsize=8)

#--- Creating colorbar ---
cbar = plt.colorbar(p,ticks=np.arange(minPower,maxPower+1,1),fraction=0.12,pad=0.02)
cbar.ax.tick_params(labelsize=8)

#--- Plotting 1D Histograms ---
xAxHist = fig.add_axes([0.3,0.05,0.6,0.15])
xAxHist.bar(unlabeledX[:-1],histXLog-minPower,width=1.0)
barColorsX = []
for ii in range(0,len(histXLog)):
    cbPercent = (histXLog[ii]-minPower)*(1.0/abs(minPower))
    barColorsX.append(pylab.cm.Spectral_r(cbPercent))
    xAxHist.bar(unlabeledX[ii],histXLog[ii]-minPower,width=1.0,color=barColorsX[ii])
xAxHist.xaxis.set_visible(False)
xAxHist.yaxis.set_visible(False)

yAxHist = fig.add_axes([0.05,0.3,0.15,0.6])
yAxHist.barh(unlabeledY[:-1],histYLog-minPower,height=1.0)
barColorsY = []
for ii in range(0,len(histYLog)):
    cbPercent = (histYLog[ii]-minPower)*(1.0/abs(minPower))
    barColorsY.append(pylab.cm.Spectral_r(cbPercent))
    yAxHist.barh(unlabeledY[ii],histYLog[ii]-minPower,height=1.0,color=barColorsY[ii])
yAxHist.xaxis.set_visible(False)
yAxHist.yaxis.set_visible(False)

#--- Setting up interactivity ---
#interactive = showPointData(fig,axarr,pointValuesX,pointValuesY,timeValues)

#--- Show the plot! ---
plt.show()

In [ ]:
#--- Test Input - Disregard ---
hist = np.zeros((12,12))
hist[0,0] = 4
hist[0,1] = 2
hist[1,0] = 1

var1Edges = np.array([-9.,1.,11.,21.,31.,41.,51.,61.,71.,81.,91.,101.,111.])
var2Edges = np.array([-20.,20.,60.,100.,140.,180.,220.,260.,300.,340.,380.,420.,460.])

var1Values = np.array([0.,0.1,0.4,0.6,0.8,0.9,7])
var2Values = np.array([1.,3.,6.,11.,34.,55.,19.])

timeValues = np.array([60,120,180,240,300,360,420])

In [ ]:
%qtconsole

#--- Setting a few final parameters before plotting ---
plottedHist = np.zeros(histLog.shape)
xLabel = var1Name
yLabel = var2Name

#--- Setting up the figure, creating the palette for the colorbar ---
fig = plt.figure(figsize=((figureXSize*1.0)/figDPI,(figureYSize*1.0)/figDPI),dpi=figDPI)
axarr = fig.add_axes([0.3,0.3,0.7,0.6])
p = axarr.pcolor(xBinEdgeScat,yBinEdgeScat,histLog,vmin=minPower,
                 vmax=maxPower,cmap='Spectral_r')
axarr.cla()

#--- Looping through the data and only scattering as many points as specified ---
for aa in range(0,len(pointValuesX)):
    locPointValueX,locPointValueY = pointValuesX[aa],pointValuesY[aa]
    xAxisBin = np.searchsorted(xBinEdges,locPointValueX)-1
    yAxisBin = np.searchsorted(yBinEdges,locPointValueY)-1
    if(0 <= xAxisBin <= 11 and 0 <= yAxisBin <= 11):
        plotThisEntry = 1
        countInBin = hist[xAxisBin,yAxisBin]
        
        #--- If more counts in bin than plottable, randomize which ones get plotted ---
        if(countInBin > maxPlottedInBin and plottedHist[xAxisBin,yAxisBin] < maxPlottedInBin):
            prob = (1.*maxPlottedInBin-plottedHist[xAxisBin,yAxisBin])/(countInBin-plottedHist[xAxisBin,yAxisBin])
            randomNumber = np.random.rand()
            if(prob < randomNumber):
                plotThisEntry = 0

        if(plotThisEntry == 1):
            plottedHist[xAxisBin,yAxisBin] += 1
            
            if(xAxisBin == 0 and locPointValueX < xBinEdgeScat[0]):
                locPointValueX = xBinEdgeScat[0]
            if(xAxisBin == xBinEdgeLen-2 and locPointValueX > xBinEdgeScat[-1]):
                locPointValueX = xBinEdgeScat[-1]
            if(yAxisBin == 0 and locPointValueY < yBinEdgeScat[0]):
                locPointValueY = yBinEdgeScat[0]
            if(yAxisBin == yBinEdgeLen-2 and locPointValueY > yBinEdgeScat[-1]):
                locPointValueY = yBinEdgeScat[-1]
            
            logCount = histLog[xAxisBin,yAxisBin]
            cbPercent = (logCount-minPower)*(1.0/abs(minPower))
            pointColor = pylab.cm.Spectral_r(cbPercent)
            axarr.scatter(locPointValueX,locPointValueY,c=pointColor)

#--- Setting axes ---
axarr.set_xlim(xBinEdgeScat[0],xBinEdgeScat[-1])
axarr.set_xticks(xBinEdgeScat)
axarr.set_xticklabels(xBinEdges[0:xBinEdgeLen],rotation=270)
axarr.tick_params(axis='x',labelsize=8)
#axarr.set_xlabel(xLabel)
axarr.set_ylim(yBinEdgeScat[0],yBinEdgeScat[-1])
axarr.set_yticks(yBinEdgeScat)
axarr.set_yticklabels(yBinEdges[0:yBinEdgeLen],rotation=360)
axarr.tick_params(axis='y',labelsize=8)
#axarr.set_ylabel(yLabel)

#--- Creating gridlines ---
for yy in range(0,len(yBinEdgeScat)):
    axarr.axhline(y=yBinEdgeScat[yy],color='#000000')
for xx in range(0,len(xBinEdgeScat)):
    axarr.axvline(x=xBinEdgeScat[xx],color='#000000')
    
axarr.set_title('Test',fontsize=8)

#--- Creating colorbar ---
cbar = plt.colorbar(p,ticks=np.arange(minPower,maxPower+1,1),fraction=0.12,pad=0.02)
cbar.ax.tick_params(labelsize=8)

#--- Plotting 1D Histograms ---
xAxHist = fig.add_axes([0.3,0.05,0.6,0.15])
xAxHist.bar(unlabeledX[:-1],histXLog-minPower,width=1.0)
barColorsX = []
for ii in range(0,len(histXLog)):
    cbPercent = (histXLog[ii]-minPower)*(1.0/abs(minPower))
    barColorsX.append(pylab.cm.Spectral_r(cbPercent))
    xAxHist.bar(unlabeledX[ii],histXLog[ii]-minPower,width=1.0,color=barColorsX[ii])
xAxHist.xaxis.set_visible(False)
xAxHist.yaxis.set_visible(False)

yAxHist = fig.add_axes([0.05,0.3,0.15,0.6])
yAxHist.barh(unlabeledY[:-1],histYLog-minPower,height=1.0)
barColorsY = []
for ii in range(0,len(histYLog)):
    cbPercent = (histYLog[ii]-minPower)*(1.0/abs(minPower))
    barColorsY.append(pylab.cm.Spectral_r(cbPercent))
    yAxHist.barh(unlabeledY[ii],histYLog[ii]-minPower,height=1.0,color=barColorsY[ii])
yAxHist.xaxis.set_visible(False)
yAxHist.yaxis.set_visible(False)

#--- Setting up interactivity ---
#interactive = showPointData(fig,axarr,pointValuesX,pointValuesY,timeValues)

#--- Show the plot! ---
plt.show()